# 10장 인공 신경망 소개

In [31]:
print(tf.__version__)

1.14.0


사람이 새를 모고 비행기에 대한 영감을 얻었듯이, 지능적인 기계에 대한 영감을 얻기위해서는 '뇌'의 구조를 살펴보는 것이 합리적이다.  
이것이 __인공 신경망(ANN)__ 의 핵심 아이디어다.  
그러나 새를 보고 비행기에 대한 영감을 얻었다 하더라도 비행기의 날개를 펄럭 거릴 필요는 없듯이 인공 신겨망도 생물학적 뉴런에서 점점 멀어지는 양상이다.  
인공신경망은 딥러닝의 핵심이다. 인공 신경망은 다재다능하고 강력하고 확장성이 좋아서 수백만 개의 이미지 분류(구글 이미지), 음성인식 서비스의 성능을 높이는 것(Siri), 수억명의 사용자에게 알맞는 비디오를 추천해주는 것(Youtube), 바둑의 수백만개의 기보를 익히고 자기자신과 게임하면서 학습하는 것(알파고) 등 아주 복잡한 대규모 머신러닝 문제를 다루는데 적합하다.  


_10장은 인공 신경망의 초창기 구조와 MLP(tensorflow로 구현)에 대한 내용을 담고있다._

## 10.1 생물학적 뉴런에서 인공 뉴런까지

- 신경망을 훈련시키기위한 데이터가 많다
- 크게 발전된 컴퓨터 하드웨어의 성능 덕분에 납득할만한 시간안에 대규모 신경망을 훈련시킬수 있다.
- 사소한 변화로 훈련 알고리즘이 대폭 향상되었다.
- 일부 인공 신경망의 이론상 제한이 실전에서는 크게 문제가 되지 않는다. 예를들면, local minimum문제, (실제로는 local minimum과 global minimum사이의 큰 gap이 없음)
- 인공 신경망이 투자와 진보의 선순환 구조에 들어섰다.

### 10.1.1 생물학적 뉴런

개개의 생물학적 뉴런은 단순하게 작동하지만 수십억개의뉴런으로 구성된 거대한 네트워크로 조직되어있고 각 뉴런은 보통 수천개의 다른 뉴런과 연결 되어있다. 이에 영감을얻어 인공신경망 작성.

### 10.1.2 뉴런을 사용한 논리연산

간단한 생물학적 뉴런모델로 어떤 논리 명제도 계산 할 수 있음을 보였음.(워런 맥컬록, 월터피츠) 

### 10.1.3 퍼셉트론

__퍼셉트론__ 은 가장 간단한 인공 신경망 구조 중 하나로 프랑ㅇ크 로젠블라트가 제안했다. TLU라는 조금 다른 형태의 인공 뉴런을 기반으로하고 입력과 출력이 어떤 숫자고 각각의 입력 연결은 가중치와 연관되어 있다. TLU는 입력의 가중치 합을 계산하고 step function을 적용하여 결과를 출력한다.  
퍼셉트론에서 가장 널리 사용되는 step function은 __헤비사이드 계단 함수(Heaviside step function)__ 이다. __부호 함수(Sign function)__을 사용하기도한다.  
하나의 TLU는 간단한 선형 이진 분류 문제에 사용할 수 있다.  
그 결과가 임곗값을 넘으면 양성, 그렇지 않으면 음성을 출력하는 식의 구조  
여기서 TLU를 훈련시킨다는 것은 최적의 w0, w1, w2(가중치 모수) 를 찾는 것과 같은 의미  
각 출력 뉴런의 결정 경계는 선형이므로 다른 여타 선형분류 알고리즘 처럼 복잡한 패턴을 학습하지 못한다. 하지만 훈련샘플이 선형적으로 구분 될 수 있다면 이 알고리즘은 정답에 수렴하게된다 (__퍼셉트론 수렴 이론__)

In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2,3)]
y = (iris.target ==0).astype(np.int)

per_clf = Perceptron(random_state = 42)
per_clf.fit(X,y)

y_pred = per_clf.predict([[2,0.5]])

C:\anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [5]:
y_pred

array([1])

퍼셉트론 알고리즘은 SGD와 상당히 유사하다.  
사이킷런의 Perceptron 클래스는 매개변수가 <loss = "perceptron", learning_rate = "constant", eta0 = 1, penalty = None> 인 SGDClassifier와 같다

퍼셉트론 알고리즘은 XOR과 같이 간단하지만 선형으로 분류될수 없는 문제를 해결 할 수 없었기 때문에 이에 대한 연구가 침체기에 들어갔다.  
그러나 여러 퍼셉트론을 쌓아 올려 일부 제약을 줄일 수 있다는 사실이 밝혀졌고, 이를 MLP(Multi-Layer Perceptron)이라고한다.

### 10.1.4 다층 퍼셉트론과 역전파

다층 퍼셉트론은 입력층(input layer)과 은닉층(hidden layer)이라 불리는 하나이상의 TLU 층과 마지막 층인 출력층(output layer)으로 구성되어있다. 인공 신경망의 은닉층이 2개 이상일때 __심층 신경망(deep neural network, DNN)__ 이라고한다.  
DNN을 학습시키는 알고리즘은 오랜 연구끝에 __역전파(backpropagation)__ 으로 귀결되었다.  
이는 후진 모드 자동 미분을 사용하는 경사 하강법으로 기술된다.  


알고리즘이 각 훈련 샘플을 네트워크에 주입하고 연속되는 각 층의 뉴런마다 출력을 계산한다.(예측을 만들때와 동일하게 정방향 계산)    
그런 다음 네트워크의 출력 오차를 계산한다.(기댓값과 네트워크 실제 출력의 차이)  
그리고 출력 뉴런의 오차에 마지막 은닉층의 뉴런이 얼마나 기여했는지 측정한다.  
이 역방향 과정은 오차 그래디언트를 후방으로 전파함으로써 네트워크의 모든 연결 가중치에 대한 오차 그래디언트를 효율적으로 계산한다.  
마지막으로, 앞서 계산한 오차 그래디언트를 네트워크의 모든 연결 가중치에 반영하는 경사 하강법 스텝을 수행한다.

요약하면,  
각 훈련샘플에 대해 역전파 알고리즘이 먼저 예측을 만들고, 오차를 측정하고, 그런 다음 역방향으로 각 층을 거치면서 각 연결이 오차에 기여한 정도를 게산하고, 마지막으로 이 오차가 감소하도록 가중치를 조금씩 조정한다.

이 알고리즘을 잘 작동시키기 위해 논문 저자들은 다층 퍼셉트론 구조에 중요한 변화를 주었다. Step function을 Logistic function으로 바꾼것이다.  계단 함수에는 수평선 밖에 없으니 계산할 그래디언트가 없기 때문.

로지스틱 이외에도 tanh(하이퍼볼릭 탄젠트), ReLU 등의 활성화 함수를 사용하기도 한다.

다분류 문제일때 (예를들면 MNIST)는 활성화 함수를 __소프트맥스(softmax function)__ 을 사용한다. 

## 10.2 텐서플로의 고수준 API로 다층 퍼셉트론 훈련하기

In [8]:
from sklearn.datasets import fetch_openml
Mnist = fetch_openml('mnist_784')

In [11]:
X, y = Mnist.data, Mnist.target
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [23]:
y_train, y_test = y_train.astype('int32'), y_test.astype('int32')

In [24]:
import tensorflow as tf

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes = 100, feature_columns = feature_cols)

dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)
dnn_clf.fit(X_train, y_train, batch_size = 50, steps = 40000)

W0406 15:16:52.112410  5884 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.
W0406 15:16:52.129364  5884 estimator.py:453] Using temporary folder as model directory: C:\Temp\tmptdm0w5_9
W0406 15:16:52.131359  5884 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.
W0406 15:16:52.313913  5884 deprecation.py:506] From C:\anaconda\lib\site-packages\tensorflow\python\training\adagrad.py:76: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0406 15:16:52.396653  5884 deprecation.py:323] From C:\anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\estimators\head.py:678: ModelFnOps.__new__ (from tensorflow.contrib.learn.python.learn.estimators.model_fn) is deprecated and will be removed in a future version

SKCompat()

In [25]:
from sklearn.metrics import accuracy_score
y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred['classes'])

W0406 15:19:22.680557  5884 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.
W0406 15:19:23.021683  5884 deprecation.py:323] From C:\anaconda\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


0.9519428571428571

위 고수준 텐서플로우 API의 내부를 들여다보면  
- DNNClassifier 클래스가 ReLU 활성화 함수  
- 출력층은 Softmax
- 비용함수는 CrossEntropy

## 10.3 텐서플로의 저수준 API로 심층 신경망 훈련하기

네트워크 구조를 더 상세하게 제어하고 싶다면 텐서플로우 저수준 파이썬 API를 사용하자.  

### 10.3.1 구성단계

In [34]:
import tensorflow as tf

n_inputs = 28*28 ##MNIST data
n_hidden1 = 300 ## hidden_layer1
n_hidden2 = 100 ## hidden_layer2
n_outputs = 10

In [29]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = "X")
y = tf.placeholder(tf.int32, shape = (None), name = "y")

플레이스홀더 X는 입력층, y는 출력층의 역할

In [32]:
def neuron_layer(X, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs+n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name = "bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

#### 위 코드 뜯어보기 
1. 먼저 층 이름으로 이름 범위를 만들었다. 여기에 이층에서 필요한 모든 계산 노드가 포함된다. 꼭 필요한것은 아니지만 노드가 잘 정리되어 있으면 텐서보드에서 훨씬 깔끔한 계산 그래프를 볼 수 있다.
2. 그런 다음 입력행렬의 크기에서 두 번째 차원을 사용해 입력 특성의 수를 구한다.(첫번쨰 차원은 샘플수)
3. 그다음에 나오는 세줄은 가중치 행렬을 담을 W 변수를 만드는 것이다(커널이라 칭함) 이 행렬은 각 입력과 각 뉴런 사이의 모든 연결 가중치를 담고 있는 2D 텐서이다. 그러므로 크기는 (n_inputs, n_neurons)가 된다. 이 행렬은 표준편차가 $2/\sqrt {n_{inputs} + n_{neurons}}$ 인 절단 정규 분포를 사용해 무작위로 초기화 된다. 이 표준편차를 사용하면 알고리즘이 훨씬 더 빠르게 수렴한다. 경사 하강법 알고리즘이 중단되지 않도록 대칭성을 피하기 위해 모든 은닉층의 가중치는 무작위로 초기화하는 것이 중요하다  
4. 다음 줄은 뉴런마다 하나의 편향을 갖도록 변수 b를 만들고 0으로 초기화한다. 
5. 그런 다음 $Z = X \times W + b$를 계산하기 위한 그래프를 만든다 이 벡터화된 구현은 층에 있는 모든 뉴런과 배치에 있는 모든 샘플에 대해 입력에 대한 가중치 합에 편향을 더하는 계산을 효율적으로 한방에 수행한다.$X \dot W$의 결과인 2D 행렬에 열의 개수와 같은 1D 배열을 더하면 모든 행에 1D배열이 더해지게된다 이를 __브로드 캐스팅__이라고한다.
6. 마지막으로 tf.nn.relu와 같은 activation 매개변수가 지정되어 있으면 activation(Z)를 반환하고 그렇지 않으면 그냥 Z를 반환한다.

In [35]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation = tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name = "hidden2", activation = tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name = "outputs")

마찬가지로 이름범위를 사용하여 정리, logits는 소프트맥스 활성화 함수로 들어가기전의 신경망 출력이다. 최적화 작업을 위해 소프트맥스 계산은 나중에..  
위 코드를 dense를 이용하여 좀 더 간단하게 바꿀 수 있다.

In [ ]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1", activation = tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name = "hidden2", activation = tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name = "outputs")

신경망 모델 정의를 마쳤으니 비용함수를 정의해야한다.  
sparse_softmax_cross_entropy_with_logits() 함수를 사용할 수 있다. 이 함수는 logits을 기반으로 크로스 엔트로피를 계산한다.

In [37]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")

GradientDescentOptimizer를 이용해 위 비용함수를 최소화 시키도록 모텔 파라미터를 조정해나갈것이다.

In [39]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

구성단계의 마지막은 모델을 평가하는 방법을 지정하는것,  
가장 간단한것은 역시 정확도(accuracy)

In [41]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

그리고 통상적으로 모든 변수를 초기화하는 노드를 만들고 훈련된 모델 파라미터를 디스크에 저장하기 위한 Saver 객체를 생성한다.

In [42]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

### 10.3.2 실행단계

In [45]:
X_train, X_test = X_train/255.0, X_test/255.0

In [46]:
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

some parameter

In [47]:
n_epochs = 40
batch_size = 50

batch shuffling

In [48]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

모델 훈련

In [50]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_valid)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.8972
1 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9148
2 배치 데이터 정확도: 0.9 검증 세트 정확도: 0.9294
3 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9392
4 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9426
5 배치 데이터 정확도: 0.9 검증 세트 정확도: 0.9478
6 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9534
7 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9562
8 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9582
9 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9598
10 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9618
11 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9608
12 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9632
13 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.964
14 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.966
15 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9666
16 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9644
17 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9704
18 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9686
19 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9694
20 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9694
21 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9706
22 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9706
23 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9696
24 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.969
25 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9704
26 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9722


위 코드 설명,  
텐서플로우의 세션을 열고 init 노드를 실행해서 모든 변수를 초기화한다.  
그 다음 바깥쪽 훈련 루프를 실행, 매 에포크에서 훈련데이터의 크기를 미니배치의 크기로 나눈 횟수만큼 반복한다.  
각 미니배치는 next_batch() 메서드로부터 추출되며, 현재 미니배치의 입력 데이터와 타깃을 주입하면서 훈련 연산을 실행시킨다.  
그 다음 에포크의 끝에서 마지막 미니배치와 검증 데이터를 사용해 모델을 평가하여 결과를 출력  
마지막으로 모델 파라미터를 디스크에 저장

### 10.3.3 신경망 사용하기

신경망을 훈련시키고나면 이를 이용해 예측을 만들 수 있다.

구성 단계는 재사용해도 되지만 실행 단계는 다음과 같이 수정 되어야한다

In [58]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    Z = logits.eval(feed_dict = {X: X_test[0:20].reshape(-1,784)})
    y_pred = np.argmax(Z, axis = 1)
    

In [59]:
print("예측 클래스:", y_pred)
print("진짜 클래스:", y_test[:20])

예측 클래스: [7 5 1 8 2 3 3 1 6 9 1 0 6 1 3 0 8 8 5 2]
진짜 클래스: [7 5 1 8 2 3 3 1 6 9 1 0 6 1 3 0 8 6 5 2]


## 10.4 신경망 하이퍼 파라미터 튜닝하기

신경망의 강력한 유연성은 중요한 단점이기도하다. 즉, 조절해야할 하이퍼 파라미터가 많아진다. 상상할 수 있는 어떤 __네트워크 토폴로지__ 도 사용할 수 있을 뿐만 아니라 다층 퍼셉트론조차도 층 수나 층마다의 뉴런 수, 각 층에서 사용하는 활성화 함수, 가중치 초기화 방식등을 바꿀 수 있다. 어떤 하이퍼파라미터 조합이 최적인지 어떻게 알 수 있을까..?

앞선 장에서 한것처럼 적절한 하이퍼파라미터를 찾기 위해 교차 검증을 활용한 그리드 탐색을 할 수 있다. 하지만 조절해야할 하이퍼파라미터가 많고 대규모 데이터셋에 신경망을 훈련할 때 오랜 시간이 걸리기 때문에 주어진 시간안에 전체 하이퍼파라미터 공간 중 작은 부분만 탐색이 가능하다. 이럴땐 RandomSearch를 사용하는 것이 낫다. 또는 좋은 하이퍼파리미터 조합을 빠르게 찾아주는 오스카 같은 도구를 사용하면 된다.

### 10.4.1 은닉층의 수

많은 문제가 은닉층 하나로 시작해도 쓸만한 결과를 얻을 수 있다. 사실 은닉층이 하나인 다층 퍼셉트론이라도 뉴런 수가 충분하면 아주 복잡한 함수도 모델링 할 수 있다는 것이 밝혀졌다.  
하지만 심층 신경망이 얕은 신경망보다 파라미터 효율성이 훨씬 좋다는 것을 간과한 사실이다. 심층 신경망은 복잡한 함수를 모델링하는데 얕은 신경망보다 훨씬 적은 수의 뉴런을 사용하기 때문에 더 빠르게 훈련된다.  


### 10.4.2 은닉층의 뉴런 수

입력층과 출력층의 뉴런 수는 해당 작업에 필요한 입력과 출력의 형태에 따라 결정된다.  예를들어 MNIST는 28 x 28 = 784개의 입력뉴런과 10개의 출력 뉴런이 필요하다. __은닉층의 구성 방식은 일반적으로 각 층의 뉴런을 점점 줄여서 깔때기 처럼 구성한다.__   
전형적인 MNIST 신경망은 첫 번째는 300개 두번째는 100개의 뉴런으로 구성된 두 개의 은닉층을 가진다.  
하지만 __요즘에는 모든 은닉층에 같은 크기를 사용하는 것이 일반적__, 예를들면 층마다 150개의 뉴런을 둔다. 즉, 하이퍼파라미터가 층마다 따로 있지 않아서 전체를 통틀어 하나만 조정하면 된다. __층의 개수와 마찬가지로 네트워크가 과대적합이 시작되기 전 까지 점진적으로 뉴런 수를 늘린다.__  불행하게도 완벽한 뉴런 수를 찾는 것은 아직도 마술 같은 일이다.. 

단순 접근 방식은 실제 필요한 것 보다 더 많은 층과 뉴런을 가진 모델을 선택하고, 그런 다음 과대적합되지 않도록 조기종료기 법을 사용하는 것이다.(혹은 드롭아웃(Dropout) 사용) 이를 __스트레치 팬츠__ 방식이라고 부른다. <나에게 맞는 사이즈를 찾느라 시간 낭비하지말고 큰 팬츠를 사고 나중에 알맞게 줄인다는 의미>

### 10.4.3 활성화 함수

대부분의 경우 은닉층에 ReLU 활성 함수 사용 이 함수는 다른 활성 함수보다 계산이 좀더 빠르고 입력값이 클때 특정 값에 수렴하지 않는 덕분에 경사 하강법이 평편한 지역에서 심하게 지체되지 않는다.  
출력층에서는 소프트맥스 활성화함수가 일반적으로 분류작업의 좋은 선택, 회귀작업일 경우에는 활성함수 사용 X